In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

# Spin-1 Fits to Data with a Constrained D/S Ratio in the Negative Reflectivity
This fit is similar to the [D/S constrained ratio study](./constrained.ipynb), but only applies the constraint to the negative reflectivity D and S waves. The positive reflectivity waves are allowed to float freely. No bootstrapping has been done for these fits yet

Fit Details:
* Only spin-1 ($J=1$) waves
  * {$J^P\ell$} = {$1^+S, 1^-P, 1^+D$}
  * $1^+(D/S)^{(-)}$ waves are constrained to each other
* Include an isotropic background term
  * Assumed that sideband subtraction handles almost all background events, and so these events will be *true* $\omega$ events, but contribute in total a flat distribution to the angular distributions
* Uses all 4 polarization orientations
* Split data into bins of mass and $-t$
  * 20 MeV wide bins from $1.0 < M_{\omega\pi^0} < 1.8$ 
* Perform 500 randomized fits in each bin of mass and $-t$ independently

## Setup

In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/data-fits/spin-1/negative-ratio/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

In [ ]:
%%bash
# print out yaml file used to submit the fits
cat $STUDY_DIR/submission.YAML

In [ ]:
# load in the preprocessed results 
# t_bins = ["t_0.10-0.16", "t_0.16-0.23", "t_0.23-0.35", "t_0.35-1.00"]
t_bins = ["t_0.10-0.16"]
binned_results = {}
for t_bin in t_bins:
    with open(f"{STUDY_DIR}/{t_bin}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
        data = pkl.load(f)
        binned_results[t_bin] = ResultManager(**data)

binned_results: dict[str, ResultManager]

In [ ]:
for t_bin, results in binned_results.items():
    print(f"Results for t bin: {t_bin}")
    results.summary()

## Analysis

### Standard Plots
These plots are always handy to have

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(30, 6))
for i, (t_bin, results) in enumerate(binned_results.items()):    
    results.plot.intensity.jp(ax=axs[i])
    t_low, t_high = results.get_t_edges()    
    axs[i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV²")
    if i != 0:
        axs[i].set_ylabel("")
        axs[i].get_legend().remove()
plt.savefig(f"{STUDY_DIR}/jp_all_t_bins.pdf")    

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.intensity.waves()
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)    
    plt.show()

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.intensity.waves(fractional=True)
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)    
    plt.show()

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.diagnostic.matrix()
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)    
    plt.show()

In [ ]:
sig_moments = set()
for t_bin, results in binned_results.items():    
    sig_moments = sig_moments.union(results.get_significant_moments(threshold=0.04))

fig, axs = plt.subplots(
    len(sig_moments), 4, figsize=(20, 25), layout="constrained", sharex=True, sharey="row"
)

row_max = {}
for col, (t_bin, results) in enumerate(binned_results.items()):    
    for row, moment in enumerate(sig_moments):        
        ax = axs[row, col]
        label = utils.convert_moment_name(moment) if col == 0 else ""
        my_kwargs = {moment: {"label": label}}
        results.plot.intensity.plot([moment], ax=ax, col_kwargs=my_kwargs)
        if col != 0:
            ax.set_ylabel("")
        max_val = ax.get_ylim()[1]
        if row not in row_max or max_val > row_max[row]:
            row_max[row] = max_val
    t_low, t_high = results.get_t_edges()
    axs[0,col].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$")


for col, (t_bin, results) in enumerate(binned_results.items()):
    for row, moment in enumerate(sig_moments):
        axs[row, col].set_ylim(top=row_max[row])
    

plt.savefig(f"{STUDY_DIR}/significant_moments_all_t_bins.pdf")


### D/S Ratio and Phase

In [ ]:
fig, axs = plt.subplots(
    1,
    3,    
    figsize=(15, 5),
    gridspec_kw={"wspace": 0.1, "hspace": 0.12},
    layout="constrained",
)
for i, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.diagnostic.ds_ratio(axs = axs)
    if i != 0:
        for ax in axs:
            ax.get_legend().remove()
    axs[-1].remove()
    axs[0].set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")
    axs[1].set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")
plt.savefig(f"{STUDY_DIR}/ds_ratio_all_t_bins.pdf", bbox_inches="tight")

In [ ]:
fig, axs = plt.subplots(
    3,
    4,
    sharex=True,
    figsize=(20, 10),
    gridspec_kw={"wspace": 0.0, "hspace": 0.12},
    layout="constrained",
)
for i, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.diagnostic.ds_ratio(axs = axs[:, i], exclude_waves=["m1p0S", "m1ppS", "m1pmS"], force_free_plotting=True)
plt.savefig(f"{STUDY_DIR}/ds_ratio_positive_all_t_bins.pdf")

In [ ]:
for pos_excluded_waves in [["p1ppS", "p1pmS"], ["p1p0S", "p1ppS"], ["p1p0S", "p1pmS"]]:
    fig, axs = plt.subplots(
        3,
        4,
        sharex=True,
        figsize=(20, 10),
        gridspec_kw={"wspace": 0.0, "hspace": 0.12},
        layout="constrained",
    )
    for i, (t_bin, results) in enumerate(binned_results.items()):
        results.plot.diagnostic.ds_ratio(axs = axs[:, i], exclude_waves=["m1p0S", "m1ppS", "m1pmS"] + pos_excluded_waves, force_free_plotting=True)
    plt.show()

### $b_1(1235)$ and $1^{--}$ interference

In [ ]:
fig, axs = plt.subplots(
    2,
    4,
    sharex=True,        
    gridspec_kw={"wspace": 0.0, "hspace": 0.07},
    height_ratios=[3, 1],
    figsize=(30, 10),
    layout="constrained",
)
for i, (t_bin, results)in enumerate(binned_results.items()):
    colors = plt.colormaps["Dark2"].colors # type: ignore # match colors to JP plot

    results.plot.phase.mass_phase(
        amp1="p1p0S", amp2="p1mpP",
        amp1_kwargs={"color":colors[2], "alpha": 1.0},
        amp2_kwargs={"color":colors[3], "alpha": 1.0},
        amp_ax=axs[0, i],
        phase_ax=axs[1, i],
    )
    t_low, t_high = results.get_t_edges()
    axs[0,i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
    if i != 0:
        axs[0,i].set_ylabel("")
        axs[1,i].set_ylabel("")
        axs[0,i].get_legend().remove()
    

max_y=0
for i in range(4):
    y_max = axs[0,i].get_ylim()[1]
    max_y = max(max_y, y_max)
for i in range(4):
    axs[0,i].set_ylim(0, max_y)


plt.savefig(f"{STUDY_DIR}/mass_phase_all_t_bins.pdf")

### Naturalities

In [ ]:
fig, axs = plt.subplots(
    4, 4, figsize=(20, 15), layout="constrained", sharex=True, sharey="row"
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    row_values = ["p1p", "p1m", "m1p", "m1m"]
    labels = ["Natural $1^+$", "Natural $1^-$", "Unnatural $1^+$", "Unnatural $1^-$"]
    colors = ["tab:red", "tab:red", "tab:blue", "tab:blue"]
    for row in range(4):
        ax = axs[row, col]    
        label = labels[row] if col == 0 else ""
        val = row_values[row]
        my_kwargs = {val: {"label": label, "color": colors[row]}}
        results.plot.intensity.plot([val], ax=ax, col_kwargs=my_kwargs)

plt.savefig(f"{STUDY_DIR}/naturalities_all_t_bins.pdf")

## Negative Ratio Constraint Vs Unconstrained Model
Now lets compare how the model with no D/S ratio constraints in either reflectivities compares

In [ ]:
unconstrained_binned_results = {}
for t_bin in t_bins:
    with open(f"{STUDY_DIR}../unconstrained-ratio/{t_bin}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
        data = pkl.load(f)
        unconstrained_binned_results[t_bin] = ResultManager(**data)
unconstrained_binned_results: dict[str, ResultManager]

### Likelihood Comparison

In [ ]:
# [2 (real, imag) params per amp] * [2 (reflectivities)] * [3 (spin projections)] * [3 (angular momenta: S, P, D)] - [2 reference phases]
unconstrained_free_params = 2 * 2 * 3 * 3 - 2

# [2 (real, imag) params per amp] * [2 (reflectivities)] * [3 (spin projections)] * [2 (angular momenta: S, P)] - [2 reference phases]
#  + [2 (mag, phase) params for D-wave in negative reflectivity] + [2 * 3 positive reflectivity D-wave params that are still free]
constrained_free_params = (2 * 2 * 3 * 2 - 2) + (2) + (2 * 3)

fig, axs = plt.subplots(
    1, 4, figsize=(20, 5), layout="constrained", sharex=True,
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.diagnostic.lrt_pvalues(
        unconstrained_binned_results[t_bin],
        constrained_free_params,
        unconstrained_free_params,
        ax=axs[col],
    )

### L-wave Comparisons

#### Natural

In [ ]:
fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharey="row", sharex=True
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    row_wave = ["p1ppS", "p1p0S", "p1pmS"]
    row_ylabel = ["$1^+S_{1}^{(+)}$", "$1^+S_{0}^{(+)}$", "$1^+S_{-1}^{(+)}$"]
    t_low, t_high = results.get_t_edges()
    for row in range(3):
        data_masses = results.get_mass_centers()
        x_err = results.get_mass_bin_width() / 2.0
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        old_values = unconstrained_binned_results[t_bin].fit_df[row_wave[row]].values
        old_errors = unconstrained_binned_results[t_bin].fit_df[f"{row_wave[row]}_err"].values

        ax = axs[row, col]
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=old_values, 
            yerr=old_errors, 
            linestyle="", 
            marker="o", 
            color="tab:red",  
            label="Unconstrained"
        )
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker="o", 
            markerfacecolor="white",
            color="tab:red",  
            label="Constrained"
        )
        
        if col == 0:
            ax.set_ylabel(row_ylabel[row], fontsize=20, loc="center")
            ax.legend()
        if row == 0:
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)", fontsize=15, loc="right")


In [ ]:
fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharey="row", sharex=True
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    row_wave = ["p1mpP", "p1m0P", "p1mmP"]
    row_ylabel = ["$1^-P_{1}^{(+)}$", "$1^-P_{0}^{(+)}$", "$1^-P_{-1}^{(+)}$"]
    t_low, t_high = results.get_t_edges()
    for row in range(3):
        data_masses = results.get_mass_centers()
        x_err = results.get_mass_bin_width() / 2.0
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        old_values = unconstrained_binned_results[t_bin].fit_df[row_wave[row]].values
        old_errors = unconstrained_binned_results[t_bin].fit_df[f"{row_wave[row]}_err"].values

        ax = axs[row, col]
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=old_values, 
            yerr=old_errors, 
            linestyle="", 
            marker="o", 
            color="tab:red",  
            label="Unconstrained"
        )
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker="o", 
            markerfacecolor="white",
            color="tab:red",  
            label="Constrained"
        )
        
        if col == 0:
            ax.set_ylabel(row_ylabel[row], fontsize=20, loc="center")
            ax.legend()
        if row == 0:
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)", fontsize=15, loc="right")


In [ ]:
fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharey="row", sharex=True
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    row_wave = ["p1ppD", "p1p0D", "p1pmD"]
    row_ylabel = ["$1^+D_{1}^{(+)}$", "$1^+D_{0}^{(+)}$", "$1^+D_{-1}^{(+)}$"]
    t_low, t_high = results.get_t_edges()
    for row in range(3):
        data_masses = results.get_mass_centers()
        x_err = results.get_mass_bin_width() / 2.0
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        old_values = unconstrained_binned_results[t_bin].fit_df[row_wave[row]].values
        old_errors = unconstrained_binned_results[t_bin].fit_df[f"{row_wave[row]}_err"].values

        ax = axs[row, col]
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=old_values, 
            yerr=old_errors,
            linestyle="", 
            marker="o", 
            color="tab:red",  
            label="Unconstrained"
        )
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=data_values, 
            yerr= data_errors, 
            linestyle="", 
            marker="o", 
            markerfacecolor="white",
            color="tab:red",  
            label="Constrained"
        )
        
        if col == 0:
            ax.set_ylabel(row_ylabel[row], fontsize=20, loc="center")
            ax.legend()
        if row == 0:
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)", fontsize=15, loc="right")


#### Unnatural

In [ ]:
fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharey="row", sharex=True
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    row_wave = ["m1ppS", "m1p0S", "m1pmS"]
    row_ylabel = ["$1^+S_{1}^{(-)}$", "$1^+S_{0}^{(-)}$", "$1^+S_{-1}^{(-)}$"]
    t_low, t_high = results.get_t_edges()
    for row in range(3):
        data_masses = results.get_mass_centers()
        x_err = results.get_mass_bin_width() / 2.0
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        old_values = unconstrained_binned_results[t_bin].fit_df[row_wave[row]].values
        old_errors = unconstrained_binned_results[t_bin].fit_df[f"{row_wave[row]}_err"].values

        ax = axs[row, col]
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=old_values, 
            yerr=old_errors, 
            linestyle="", 
            marker="o", 
            color="tab:blue",  
            label="Unconstrained"
        )
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker="o", 
            markerfacecolor="white",
            color="tab:blue",  
            label="Constrained"
        )
        
        if col == 0:
            ax.set_ylabel(row_ylabel[row], fontsize=20, loc="center")
            ax.legend()
        if row == 0:
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)", fontsize=15, loc="right")


In [ ]:
fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharey="row", sharex=True
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    row_wave = ["m1mpP", "m1m0P", "m1mmP"]
    row_ylabel = ["$1^-P_{1}^{(-)}$", "$1^-P_{0}^{(-)}$", "$1^-P_{-1}^{(-)}$"]
    t_low, t_high = results.get_t_edges()
    for row in range(3):
        data_masses = results.get_mass_centers()
        x_err = results.get_mass_bin_width() / 2.0
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        old_values = unconstrained_binned_results[t_bin].fit_df[row_wave[row]].values
        old_errors = unconstrained_binned_results[t_bin].fit_df[f"{row_wave[row]}_err"].values

        ax = axs[row, col]
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=old_values, 
            yerr=old_errors, 
            linestyle="", 
            marker="o", 
            color="tab:blue",  
            label="Unconstrained"
        )
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker="o", 
            markerfacecolor="white",
            color="tab:blue",  
            label="Constrained"
        )
        
        if col == 0:
            ax.set_ylabel(row_ylabel[row], fontsize=20, loc="center")
            ax.legend()
        if row == 0:
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)", fontsize=15, loc="right")


In [ ]:
fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharey="row", sharex=True
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    row_wave = ["m1ppD", "m1p0D", "m1pmD"]
    row_ylabel = ["$1^+D_{1}^{(-)}$", "$1^+D_{0}^{(-)}$", "$1^+D_{-1}^{(-)}$"]
    t_low, t_high = results.get_t_edges()
    for row in range(3):
        data_masses = results.get_mass_centers()
        x_err = results.get_mass_bin_width() / 2.0
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        old_values = unconstrained_binned_results[t_bin].fit_df[row_wave[row]].values
        old_errors = unconstrained_binned_results[t_bin].fit_df[f"{row_wave[row]}_err"].values

        ax = axs[row, col]
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=old_values, 
            yerr=old_errors, 
            linestyle="", 
            marker="o", 
            color="tab:blue",  
            label="Unconstrained"
        )
        ax.errorbar(
            x=data_masses, 
            xerr=x_err,
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker="o", 
            markerfacecolor="white",
            color="tab:blue",  
            label="Constrained"
        )
        
        if col == 0:
            ax.set_ylabel(row_ylabel[row], fontsize=20)
            ax.legend()
        if row == 0:
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)", fontsize=15, loc="right")


### Total Naturalities by JP

In [ ]:
fig, axs = plt.subplots(
    4, 4, figsize=(20, 15), layout="constrained", sharex=True, sharey="row"
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    row_values = ["p1p", "p1m", "m1p", "m1m"]    
    t_low, t_high = results.get_t_edges()
    colors = ["tab:red", "tab:red", "tab:blue", "tab:blue"]

    for row in range(4):
        ax = axs[row, col]    
        label = "constrained" if col == 0 else ""
        val = row_values[row]

        my_kwargs = {val: {"label": label, "color": colors[row], "markerfacecolor":"white", "marker": "o"}}
        results.plot.intensity.plot([val], ax=ax, col_kwargs=my_kwargs)

        label = "unconstrained" if col == 0 else ""
        my_kwargs = {val: {"label": label, "color": colors[row], "marker":"o"}}
        unconstrained_binned_results[t_bin].plot.intensity.plot(
            [val], ax=ax, col_kwargs=my_kwargs,
        )

        if col == 0:
            ax.set_ylabel(utils.convert_amp_name(row_values[row]), fontsize=20, loc="center")
            ax.legend()
        else:
            ax.set_ylabel("")
        if row == 0:
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)
        else:
            ax.set_title("")
        if row == 3:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)", fontsize=15, loc="right")
        else:
            ax.set_xlabel("")

### $b_1(1235)$ and $1^{--}$ interference

In [ ]:
fig, axs = plt.subplots(
    2,
    4,
    sharex=True,        
    gridspec_kw={"wspace": 0.0, "hspace": 0.07},
    height_ratios=[3, 1],
    figsize=(30, 10),
    layout="constrained",
)
for i, (t_bin, results)in enumerate(binned_results.items()):
    colors = plt.colormaps["Dark2"].colors # type: ignore # match colors to JP plot

    results.plot.phase.mass_phase(
        amp1="p1p0S", amp2="p1mpP",
        amp1_kwargs={"color":colors[2], "alpha": 1.0, "markerfacecolor":"white", "marker":"o"},
        amp2_kwargs={"color":colors[3], "alpha": 1.0, "markerfacecolor":"white", "marker":"o"},
        phase_kwargs={"marker":"o", "markerfacecolor":"white"},
        amp_ax=axs[0, i],
        phase_ax=axs[1, i],
    )
    unconstrained_binned_results[t_bin].plot.phase.mass_phase(
        amp1="p1p0S", amp2="p1mpP",
        amp1_kwargs={"color":colors[2], "alpha": 1.0, "marker":"o"},
        amp2_kwargs={"color":colors[3], "alpha": 1.0, "marker":"o"},
        phase_kwargs={"marker":"o"},
        amp_ax=axs[0, i],
        phase_ax=axs[1, i],
    )

    t_low, t_high = results.get_t_edges()
    axs[0,i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
    if i != 0:
        axs[0,i].set_ylabel("")
        axs[1,i].set_ylabel("")
        axs[0,i].get_legend().remove()
    

max_y=0
for i in range(4):
    y_max = axs[0,i].get_ylim()[1]
    max_y = max(max_y, y_max)
for i in range(4):
    axs[0,i].set_ylim(0, max_y)


plt.savefig(f"{STUDY_DIR}/ratio_compare_mass_phase_all_t_bins.pdf")